In [1]:
import numpy as np
import tensorflow as tf
from adult_modified import preprocess_adult_data
from sklearn import linear_model
import utils

In [2]:
seed = 1
dataset_orig_train, dataset_orig_test = preprocess_adult_data(seed = seed)

In [3]:
x_unprotected_train, x_protected_train = dataset_orig_train.features[:, :39], dataset_orig_train.features[:, 39:]
x_unprotected_test, x_protected_test = dataset_orig_test.features[:, :39], dataset_orig_test.features[:, 39:]
y_train, y_test = dataset_orig_train.labels.reshape((-1,)), dataset_orig_test.labels.reshape((-1,))



In [4]:
x_unprotected_train, x_unprotected_test = tf.cast(x_unprotected_train, dtype = tf.float32), tf.cast(x_unprotected_test, dtype = tf.float32)


In [8]:
y_train, y_test = tf.one_hot(y_train, 2), tf.one_hot(y_test, 2)

NotFoundError: Could not find valid device for node.
Node:{{node OneHot}}
All kernels registered for op OneHot :
  device='XLA_CPU_JIT'; TI in [DT_INT32, DT_UINT8, DT_INT64]; T in [DT_FLOAT, DT_DOUBLE, DT_INT32, DT_UINT8, DT_INT16, ..., DT_UINT16, DT_COMPLEX128, DT_HALF, DT_UINT32, DT_UINT64]
  device='XLA_CPU'; TI in [DT_INT32, DT_UINT8, DT_INT64]; T in [DT_FLOAT, DT_DOUBLE, DT_INT32, DT_UINT8, DT_INT16, ..., DT_UINT16, DT_COMPLEX128, DT_HALF, DT_UINT32, DT_UINT64]
  device='CPU'; TI in [DT_UINT8]; T in [DT_INT64]
  device='CPU'; TI in [DT_INT32]; T in [DT_INT64]
  device='CPU'; TI in [DT_INT64]; T in [DT_INT64]
  device='CPU'; TI in [DT_UINT8]; T in [DT_INT32]
  device='CPU'; TI in [DT_INT32]; T in [DT_INT32]
  device='CPU'; TI in [DT_INT64]; T in [DT_INT32]
  device='CPU'; TI in [DT_UINT8]; T in [DT_UINT16]
  device='CPU'; TI in [DT_INT32]; T in [DT_UINT16]
  device='CPU'; TI in [DT_INT64]; T in [DT_UINT16]
  device='CPU'; TI in [DT_UINT8]; T in [DT_INT16]
  device='CPU'; TI in [DT_

In [7]:
graph = utils.ClassifierGraph(100, 2)



def Classifier(graph, x_train, y_train, x_test, y_test, num_steps = 10000, batch_size = 250, learning_rate = 1e-4):
    batch_data = []
    # Tensor slice for train data
    batch = tf.data.Dataset.from_tensor_slices((x_unprotected_train, y_train))
    batch = batch.repeat().shuffle(5000).batch(batch_size)
    batch_data.append(batch.take(num_steps))

    # Tensor slice for test data
    batch = tf.data.Dataset.from_tensor_slices((x_unprotected_test, y_test))
    batch = batch.repeat().shuffle(5000).batch(batch_size)
    batch_data.append(batch.take(num_steps))

    # Adam optimizer
    optimizer = tf.optimizers.Adam(learning_rate)

    def train_step(data_train_epoch, step):
        x, y = data_train_epoch
        with tf.GradientTape() as g:
            loss = utils.EntropyLoss(y, graph(x, training = True))

        variables = graph.trainable_variable
        gradients = g.gradients(loss, variables)
        optimizer.apply_gradients(zip(gradients, variables))
    
    for step, data in enumerate(zip(*batch_data), 1):
        batch_data_train, _ = data
        train_step(batch_data_train, step)
    
    return graph


In [9]:
prob = graph(x_unprotected_train, training = True)

In [17]:
prob.shape

TensorShape([36177, 2])

In [18]:
y_train.shape

TensorShape([36177, 1])

In [7]:
res = tf.one_hot([0,3], 2)